# Process for PlanetScope Image Download

### Considerations:

1. The image was already downloaded from the PlanetScope repository and uploaded to Google Cloud Earth Engine Asset.
2. This was done to avoid the high time in calculating the indexes; it is more efficient to do this using geemap and/or Python.

### Index 

### 1. Prepare the image for download
### 2. The image exported to Drive needs to be downloaded from Drive to the local machine

In [ ]:
# Import the libraries
import geemap
import ee

In [2]:
# Authenticate your GEE account
ee.Authenticate()

True

In [5]:
# Importar librerias al entorno
import ee

# Iniciar Sesión en GEE (si les da un error incluso pegando el link de autenticación hay que conectar GEE con Google Cloud)
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [64]:
# Add a map to display the image and index
Map = geemap.Map(center = (39.22, -8.97), zoom = 6)
Map

Map(center=[39.22, -8.97], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [65]:
Map.user_roi.getInfo()
#Define the geometry of the study area
geometry = Map.user_roi

In [66]:
Map.user_roi.getInfo()

{'type': 'Point', 'coordinates': [-8.607292, 38.134876]}

## The points of each downloaded area are:
1. Quercus_1: geometry = ee.Geometry.Point([-8.796394, 39.205319])
2. Quercus_2: geometry = ee.Geometry.Point([-8.82684, 38.964544])
3. Quercus_3: geometry = ee.Geometry.Point([-9.130675, 38.625997])
4. Quercus_4: geometry = ee.Geometry.Point([-7.688382, 38.655686])
5. Quercus_5: geometry = ee.Geometry.Point([-8.515074, 38.333295])
6. Quercus_6: geometry = ee.Geometry.Point([-8.548938, 38.175632])
7. Quercus_7: geometry = ee.Geometry.Point([-8.727648, 38.118924])
8. Quercus_8: geometry = ee.Geometry.Point([-8.73125, 38.092178])
9. Quercus_9: geometry = ee.Geometry.Point([-8.510705, 37.186572])
10. Quercus_10: geometry = ee.Geometry.Point([-8.691703, 37.92617])
11. Quercus_11: geometry = ee.Geometry.Point([-8.607292, 38.134876])

In [67]:
# Asegurarnos de la creación de una geometría / Por si no la crearon en el mapa anterior.
geometry = ee.Geometry.Point([-8.607292, 38.134876])


In [ ]:
# Call the image collection
collection = ee.ImageCollection("projects/ee-jesusc461/assets/Planet_quercus/Quercus_11")

# Filter by date (in this case is the same month of soil sampling)
collection = collection.filterDate('2023-05-01', '2024-05-31').filterBounds(geometry) 

# This function maps spectral indices for salinity Mapping using PlanetScope Imagery
def addIndices(img):
    # NDVI
    NDVI = img.normalizedDifference(['nir', 'red']).rename('NDVI')
    
    # SR (Simple Ratio)
    SR = img.select('nir').divide(img.select('red')).rename('SR')
    
    # GCVI (Green Chlorophyll Vegetation Index)
    GCVI = img.expression('(NIR / GREEN) - 1', {'NIR': img.select('nir'), 'GREEN': img.select('green')}).rename('GCVI')
    
    # NDWI (Normalized Difference Water Index)
    NDWI = img.expression('(GREEN - NIR) / (GREEN + NIR)', {'NIR': img.select('nir'), 'GREEN': img.select('green')}).rename('NDWI')
    
    # VARI (Visible Atmospherically Resistant Index)
    VARI = img.expression('(Green - Red) / (Green + Red - Blue)', {'Blue': img.select('blue'), 'Red': img.select('red'), 'Green': img.select('green')}).rename('VARI')
    
    # GRVI (Green Ratio Vegetation Index)
    GRVI = img.expression('(NIR / GREEN)', {'NIR': img.select('nir'), 'GREEN': img.select('green')}).rename('GRVI')
    
    # GNDVI (Green Normalized Difference Vegetation Index)
    GNDVI = img.normalizedDifference(['nir', 'green']).rename('GNDVI')
    
    # SAVI (Soil Adjusted Vegetation Index)
    SAVI = img.expression('(NIR - RED) / ((NIR + RED + 0.5) * 1.5)', {'NIR': img.select('nir'), 'RED': img.select('red')}).rename('SAVI')
    
    # ARVI (Atmospherically Resistant Vegetation Index)
    ARVI = img.expression(
        '(NIR - (RED - 1.0 * (RED - BLUE))) / (NIR + (RED - 1.0 * (RED - BLUE)))',
        {'NIR': img.select('nir'), 'RED': img.select('red'), 'BLUE': img.select('blue')}
    ).rename('ARVI')
    
    # EVI (Enhanced Vegetation Index)
    EVI = img.expression(
        '2.5 * ((NIR - RED) / (NIR + 6.0 * RED - 7.5 * BLUE + 1.0))',
        {'NIR': img.select('nir'), 'RED': img.select('red'), 'BLUE': img.select('blue')}
    ).rename('EVI')
    
    # DVI (Difference Vegetation Index)
    DVI = img.expression('NIR - RED', {'NIR': img.select('nir'), 'RED': img.select('red')}).rename('DVI')

    return img.addBands([NDVI, SR, GCVI, NDWI, VARI, GRVI, GNDVI, SAVI, ARVI, EVI, DVI])

# Example usage
ps = collection
#Add the indices
ps = ps.map(addIndices)
composite = ps \
              .mean()
              
Map.addLayer(composite, {'bands': ['red',  'green',  'blue'], 'min': 201, 'max': 2464}, 'RGB')

In [69]:
# Import necessary libraries
import ee

# Ensure Earth Engine is initialized
ee.Initialize()

# Define the export function for Google Drive
def export_to_drive(image_collection, folder_name="Quercus_11"):
    # Iterate over each image in the collection
    image_list = image_collection.toList(image_collection.size())  # Convert collection to list for iteration
    num_images = image_collection.size().getInfo()  # Get the number of images

    for i in range(num_images):
        # Select the image by index and convert it to float32
        img = ee.Image(image_list.get(i)).toFloat()  # Convert image to float32
        
        # Get the image ID or a unique identifier for naming
        img_id = img.id().getInfo() if img.id().getInfo() else f"image_{i + 1}"

        # Set up export task
        task = ee.batch.Export.image.toDrive(
            image=img,
            description=f"Export_{img_id}",
            folder=folder_name,
            fileNamePrefix=f"{img_id}_indices",
            scale=3,  # 3m for PlanetScope
            maxPixels=1e13
        )
        
        # Start the export task
        task.start()
        print(f"Started export task for {img_id}")

# Call the export function with the processed collection
export_to_drive(ps)


Started export task for 2023-05-07_strip_6489944_composite
Started export task for 2023-05-07_strip_6490009_composite
Started export task for 2023-05-10_strip_6497430_composite
Started export task for 2023-05-12_strip_6502573_composite
Started export task for 2023-05-13_strip_6504891_composite
Started export task for 2023-05-14_strip_6507361_composite
Started export task for 2023-05-14_strip_6508079_composite
Started export task for 2023-05-15_strip_6509880_composite
Started export task for 2023-05-15_strip_6510066_composite
Started export task for 2023-05-19_strip_6519238_composite
Started export task for 2023-06-14_strip_6577873_composite
Started export task for 2023-06-17_strip_6584624_composite
Started export task for 2023-06-22_strip_6596271_composite
Started export task for 2023-06-25_strip_6603163_composite
Started export task for 2023-06-28_strip_6610360_composite
Started export task for 2023-06-29_strip_6612834_composite
Started export task for 2023-07-03_strip_6621765_composi